In [1]:
from syllables import *
import os
import copy
import sys
from functools import reduce
import string
import random
import shutil
import functools

Loaded syllables.py


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, XLNetTokenizer, XLNetLMHeadModel
from nltk.corpus import cmudict
import torch

cmu_save = cmudict.entries()
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
xlnet_model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased")


In [3]:
### Top-k most frequent rhymes
RHYME_K = 50                          
# for top-k sampling of tokens from language models
SAMPLE_K = 40                         
# How many times to try a line
NUM_TRIES = 6                         
### Always pick the best scoring line?
GREEDY_PICK_LINE = False
### temperature for picking different possible options for lines. Set to 1 to be most permissive. 
### Set closer to zero to be more conservative and prefer the best scoring line more.
PICK_LINE_TEMPERATURE = 0.1
### Setting this too high can cause memory problems
MAX_CONTEXT_LENGTH = 125

In [4]:
### These are legit letters
STR_LETTERS = set(string.ascii_letters + string.digits)

### Is a string a word?
def is_word(str):
  str_set = set(str)
  return len(str) > 0 and str_set.issubset(STR_LETTERS)

### Is a letter a vowel?
def is_vowel(letter):
  return letter in ['a', 'e', 'i', 'o', 'u', 'y']

### Is a letter a consonant?
def is_consonant(letter):
  return not is_vowel(letter)

### Is a phoneme a vowel sound?
### Pass in a phoneme (get this from Phonetic.phones_list)
def is_vowel_sound(phoneme):
  return is_vowel(phoneme[0])# and (is_vowel(phoneme[-1]) or phoneme[-1] in ['h', 'x', 'y'])

### Is a phoneme a consonant sound?
### Pass in a phoneme (get this from Phonetic.phones_list)
def is_consonant_sound(phoneme):
  return not is_vowel_sound(phoneme)

### Return the number of vowel phonemes in a phoneneme list.
### Pass in a list of phonemes (get this from Phonetic.phones_list)
def num_vowel_phones(phone_list):
  count = 0
  for ph in phone_list:
    if is_vowel(ph[0]):
      count = count + 1
  return count

### For whatever reason, the first call to phonemize crashes because the API call fails.
### This calls phonemize() once with a burner word just to get it out of the system
def test_phonemize():
  word = 'foobar'
  try:
    festival = phonemize(word, separator=Separator(phone='.', syllable='|', word=' ')).strip()
  except:
    pass
  try:
    espeak = phonemize(word, backend='espeak', with_stress=True, separator=Separator(phone='.', syllable='', word=' ')).strip()
  except:
    pass

### Return the indicies of a character (ch) in a string (s)
def find_char_indexes(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

### Gets the phonemize API going
test_phonemize()

In [5]:
%pip install phonemizer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from phonemizer.phonemize import phonemize
from phonemizer.separator import Separator

### Phonetic class
### word: orignal word
class Phonetic:
    def __init__(self, word):
        global PHONE_CACHE
        self.word = word  # Remember the word
        festival = None  # festival results
        espeak = None  # espeak results
        # If the word is in cache, use those results
        if word in PHONE_CACHE:
            festival, espeak = PHONE_CACHE[word]
        else:
            # API calls
            festival = phonemize(
                word, separator=Separator(phone=".", syllable="|", word=" ")
            ).strip()
            espeak = phonemize(
                word,
                backend="espeak",
                with_stress=True,
                separator=Separator(phone=".", syllable="", word=" "),
            ).strip()
            # Store the results
            PHONE_CACHE[word] = (festival, espeak)
        # List of syllables. Each syllable is a list of phones
        self.syllables_with_phones = [
            syl[:-1].split(".") for syl in festival.split("|")[:-1]
        ]
        # Just the syllables in raw form
        self.syllables = festival.replace(".", "")
        # Just the syllables in list form
        self.syllables_list = self.syllables.split("|")[:-1]
        # Just the phones in raw form
        self.phones = festival.replace("|", "")
        # Just the phones in list form
        self.phones_list = self.phones.split(".")[:-1]
        self.major_stress = 0  # The major stresses phone (index)
        self.minor_stresses = []  # List of minor stressed phones (list of indices)
        # Compute the major stress
        major_stress_char_idxs = find_char_indexes(espeak, "ˈ")
        if len(major_stress_char_idxs) > 0:
            stress_idx = espeak[: major_stress_char_idxs[0]].count(".")
            if stress_idx < len(self.phones_list):
                self.major_stress = stress_idx
            else:
                # Find the next earliest vowel phone
                for i in range(len(self.phones_list)):
                    if is_vowel_sound(self.phones_list[len(self.phones_list) - i - 1]):
                        self.major_stress = len(self.phones_list) - i - 1
                        break
        # compute the minor stresses
        minor_stress_char_idxs = find_char_indexes(espeak, "ˌ")
        if len(minor_stress_char_idxs) > 0:
            for char_idx in minor_stress_char_idxs:
                stress_idx = espeak[:char_idx].count(".")
                if stress_idx < len(self.phones_list):
                    self.minor_stresses.append(stress_idx)
                else:
                    # Find the next earliest vowel phone
                    for i in range(len(self.phones_list)):
                        if is_vowel_sound(
                            self.phones_list[len(self.phones_list) - i - 1]
                        ):
                            self.minor_stresses.append(len(self.phones_list) - i - 1)
                            break
            # all stressed phones (list of indices)
        self.all_stresses = sorted([self.major_stress] + self.minor_stresses[:])

    def num_phones(self):
        return len(self.phones_list)

    def num_syllables(self):
        return len(self.syllables_list)

    def get_num_vowels(self):
        return num_vowel_phones(self.phones_list)

    def get_nth_vowel_phone(self, n=0):
        count = -1
        for i, ph in enumerate(self.phones_list):
            if is_vowel_sound(ph):
                count = count + 1
            if count == n:
                return ph, i
        else:
            return None

    def get_syllable_of_nth_phone(self, n):
        count = -1
        for i, syl in enumerate(self.syllables_list):
            for ph in syl:
                count = count + 1
                if count == n:
                    return i
        return None

In [7]:
### Return true if two words are perfect ryles of each other
def perfect_rhyme(word1, word2):
  phonetic1 = Phonetic(word1)
  phonetic2 = Phonetic(word2)
  stress1 = phonetic1.all_stresses[-1]
  stress2 = phonetic2.all_stresses[-1]
  if phonetic1.num_phones() - stress1 != phonetic2.num_phones() - stress2:
    return False
  for i in range(phonetic1.num_phones() - stress1):
    phone1 = phonetic1.phones_list[i + stress1]
    phone2 = phonetic2.phones_list[i + stress2]
    if phone1 != phone2:
      return False
  return True

### Let's pretend that these phones are the same
### (In addition all vowel phones that start with the same letter will be considered the same)
NEAR_SETS = [['er', 'r'], 
             #['ih', 'eh'],
             ['eh', 'ah'],
             ['er', 'ax'],
             ['ae', 'ey'],
             ['t', 'f'],
             ['b', 'k'],
             ['p', 'z', 'th'],
             ['s', 'st', 'sk'],
             ['nt', 'ns'],
             ['n', 'ng']
             ]

### These phonemes start with the same letter but should not be considered eqivalent
FAR_SETS =[['aw', 'ay', 'ax'],
           ['ax', 'ao']]

### Determine if phonemes are a near match
def near_match(phone1, phone2):
  # Safety check
  if (phone1 is None) or (phone2 is None) or (len(phone1) == 0 and len(phone2) > 0) or (len(phone2) == 0 and len(phone1) > 0):
    return False
  if phone1 == phone2:
    # Phones are the same
    return True
  elif is_vowel_sound(phone1) and is_vowel_sound(phone2) and phone1[0] == phone2[0]:
    # Vowel sounds start with the same letter
    # Check that they are not in the same far set
    for fs in FAR_SETS:
      if phone1 in fs and phone2 in fs:
        return False
    return True
  else:
    # Check to see if the pair of phones are in the same near_set
    for ns in NEAR_SETS:
      if phone1 in ns and phone2 in ns:
        return True
  return False

### Check if every phone in list 1 is a near match to corresponding phone in list 2
def near_matches(phone_list1, phone_list2):
  if len(phone_list1) != len(phone_list2):
    return False
  else:
    for i in range(len(phone_list1)):
      if not near_match(phone_list1[i], phone_list2[i]):
        return False
  return True

In [8]:
### If you turn this on, it will print information about the phoneme analysis being conducted
### every time near_rhyme() is called. Not recommended
VERBOSE = False


### Verbose print allows for debug printing to be turned off
def vprint(*args):
    if VERBOSE:
        print(" ".join([str(a) for a in args]))


### Determine if word1 and word2 are near rhymes
### if last_consonant is False, we won't check for near matches of the last consonant
def near_rhyme(word1, word2, last_consonant=True):
    if perfect_rhyme(word1, word2):
        return True
    phonetic1 = Phonetic(word1)
    phonetic2 = Phonetic(word2)
    stress1 = phonetic1.all_stresses[-1]  # The last stressed phone
    stress2 = phonetic2.all_stresses[-1]  # The last stressed phone
    vprint(phonetic1.syllables_with_phones, phonetic1.all_stresses)
    vprint(phonetic2.syllables_with_phones, phonetic2.all_stresses)

    # Word = a v x w c
    # a = prefix (not important)
    # v = last stressed vowel
    # x = any number of phones between v and w
    # w = last vowel
    # c = consonant after last vowel

    # Last stressed vowel (v)
    v1_index = stress1
    v2_index = stress2
    v1 = phonetic1.phones_list[v1_index]
    v2 = phonetic2.phones_list[v2_index]
    vprint("v:", v1, v1_index, v2, v2_index)

    # Last vowel (w)
    w1, w1_index = phonetic1.get_nth_vowel_phone(phonetic1.get_num_vowels() - 1)
    w2, w2_index = phonetic2.get_nth_vowel_phone(phonetic2.get_num_vowels() - 1)
    vprint("w:", w1, w1_index, w2, w2_index)

    # Consonants after last vowel (c)
    c1_index = w1_index + 1
    c1 = "".join(phonetic1.phones_list[c1_index : phonetic1.num_phones()])
    c2_index = w2_index + 1
    c2 = "".join(phonetic2.phones_list[c2_index : phonetic2.num_phones()])
    vprint("c:", c1, c1_index, c2, c2_index)

    # phones between v and w (x)
    x1 = phonetic1.phones_list[v1_index + 1 : w1_index]
    x2 = phonetic2.phones_list[v2_index + 1 : w2_index]
    vprint("x:", x1, x2)

    # p = first phone in x
    # q = last phone in x
    p1 = None
    q1 = None
    p2 = None
    q2 = None
    if len(x1) > 0:
        p1 = x1[0]
        q1 = x1[-1]
    if len(x2) > 0:
        p2 = x2[0]
        q2 = x2[-1]
    vprint("p,q:", p1, q1, p2, q2)

    if not near_match(w1, w2):
        vprint("w fail")
        return False
    elif not near_match(v1, v2):
        vprint("v fail")
        return False
    elif False and len(c1) != len(c2) and (len(c1) > 1 or len(c2) > 1):
        vprint("c not same length - fail")
        return False
    elif last_consonant and not near_match(c1, c2):
        vprint("cs dont match - fail")
        return False
    elif len(x1) == 0 and len(x2) == 0:
        vprint("no x - match")
        return True
    elif len(x1) == 1 and len(x2) == 1 and near_match(x1[0], x2[0]):
        vprint("single x - match")
        return True
    elif len(x1) > 0 and len(x2) > 0 and num_vowel_phones(x1) != num_vowel_phones(x2):
        vprint("num vowel phones in x - fail")
        return False
    elif near_match(p1, p2) and near_match(q1, q2):
        vprint("ps or qs - match")
        return True
    return False

In [9]:
import os
import shutil
import urllib.request
import zipfile

NEAR_RHYME_PATH = "data/near_rhymes20000"
PHONE_CACHE_PATH = "data/phone_cache"
NEAR_RHYME_DICT_FILENAME = "data/near_rhymes20000_dict"
PHONE_CACHE_FILENAME = "data/phone_cache_dict"

if not os.path.exists(NEAR_RHYME_PATH):
    url_near_rhymes = "https://www.dropbox.com/s/8a800ivlp0uknic/near_rhymes20000.zip?dl=1"
    urllib.request.urlretrieve(url_near_rhymes, "near_rhymes20000.zip")
    with zipfile.ZipFile("near_rhymes20000.zip", "r") as zip_ref:
        zip_ref.extractall(NEAR_RHYME_PATH)
    os.remove("near_rhymes20000.zip")

if not os.path.exists(PHONE_CACHE_PATH):
    url_phone_cache = "https://www.dropbox.com/s/i57hvmnlint7wj3/phone_cache.zip?dl=1"
    urllib.request.urlretrieve(url_phone_cache, "phone_cache.zip")
    with zipfile.ZipFile("phone_cache.zip", "r") as zip_ref:
        zip_ref.extractall(PHONE_CACHE_PATH)
    os.remove("phone_cache.zip")

if not os.path.exists(NEAR_RHYME_PATH):
    shutil.copyfile(NEAR_RHYME_DICT_FILENAME, NEAR_RHYME_PATH)

if not os.path.exists(PHONE_CACHE_PATH):
    shutil.copyfile(PHONE_CACHE_FILENAME, PHONE_CACHE_PATH)


In [10]:
### Load the phone cache from disk
def load_phone_cache(filename):
    cache = {}
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            split_line = line.split("\t")
            if len(split_line) >= 3:
                word, festival, espeak = split_line
                cache[word] = (festival, espeak)

        return cache
    return cache


def load_near_rhyme_dictionary(filename):
    rhyme_dict = {}
    with open(filename, "r") as f:
        for line in f.readlines():
            line = line.split("\t")
            key = line[0].strip()
            val = line[1].strip()
            if key not in rhyme_dict:
                rhyme_dict[key] = []
            rhyme_dict[key].append(val)
        return rhyme_dict
    return rhyme_dict

In [11]:
if os.path.exists(NEAR_RHYME_PATH):
    NEAR_RHYME_DICT = load_near_rhyme_dictionary(NEAR_RHYME_PATH + "/near_rhymes20000")

if os.path.exists(PHONE_CACHE_PATH):
    PHONE_CACHE = load_phone_cache(PHONE_CACHE_PATH + "/phone_cache")

In [12]:
### Make a dictionary remembering whether words are near rhymes to each other.
def make_near_rhyme_dictionary(top_n, filename, rhyme_dict=None):
    global VERBOSE
    VERBOSE = False  # Turn off debugging prints
    near_rhymes = []
    # Get top n most frequent words
    top = top_n_list("en", top_n, wordlist="best")
    for i, w1 in enumerate(top):
        for j, w2 in enumerate(top):
            if is_word(w1) and is_word(w2) and w1 != w2:
                if (
                    rhyme_dict is None
                    or w1 not in rhyme_dict
                    or w2 not in rhyme_dict[w1]
                ):
                    if near_rhyme(w1, w2):
                        print(
                            "MATCH",
                            i,
                            j,
                            w1,
                            w2,
                        )
                        near_rhymes.append((w1, w2))
    if rhyme_dict is not None:
        for key in list(rhyme_dict.keys()):
            for val in rhyme_dict[key]:
                near_rhymes.append((key, val))
                near_rhymes.append((val, key))
    with open(filename, "w") as f:
        for w1, w2 in near_rhymes:
            f.write(w1 + "\t" + w2 + "\n")

In [13]:
gpt2_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [14]:
xlnet_model.eval()

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (l

In [15]:
### Are GPUs available?
CUDA_AVAILABLE = torch.cuda.is_available()

# Which model is in the GPU (string)
MODEL_IN_GPU = None
# Which models are we using and what are their names?
MODEL_HASH = {"gpt": gpt2_model, "xlnet": xlnet_model}


def _prep_model(model_name):
    global MODEL_IN_GPU, MODELS_NOT_IN_GPU
    if MODEL_IN_GPU != model_name:
        # Unload the model in the gpu (if any)
        if MODEL_IN_GPU is not None:
            MODEL_HASH[MODEL_IN_GPU].to("cpu")
            MODEL_IN_GPU = None
        # Load the new model to gpu
        if CUDA_AVAILABLE:
            print("LOADING", model_name)
            MODEL_HASH[model_name].to("cuda")
            MODEL_IN_GPU = model_name


# For backward compatibility:
def prep_gpt():
    print("Using GPT")
    _prep_model("gpt")


def prep_xlnet():
    print("Using XLNet")
    _prep_model("xlnet")


# Decorators!
def use_gpt(func):
    def wrapper(*args, **kwargs):
        _prep_model("gpt")
        return func(*args, **kwargs)

    return wrapper


def use_xlnet(func):
    def wrapper(*args, **kwargs):
        _prep_model("xlnet")
        return func(*args, **kwargs)

    return wrapper

In [16]:
### This does top-k and top-p sampling from a list of logits. Borrowed from original GPT-2 code.
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [17]:
### These are punctuation
from string import punctuation as PUNCTUATION
from string import digits as NUMBERS

### Periods are important
PERIOD = "."
BLANK = " "


### Is this string punctuation?
def is_punctuation(s):
    return len(set(s).intersection(set(PUNCTUATION))) > 0


### Remove punctuation from string
def remove_punctuation(s):
    return "".join(i for i in s if not i in PUNCTUATION)


### How many syllables in this string?
def get_syllables_for_line(line):
    line = "".join(list(filter(lambda c: c in STR_LETTERS or c == BLANK, line)))
    count = 0
    words = line.split()
    for word in words:
        word = word.strip()
        if len(word) > 0 and is_word(word):
            phonetic = Phonetic(word)
            count = count + phonetic.num_syllables()
    return count


### Remove the prefix from the string (s)
def remove_prefix(s, prefix):
    rest = s[len(prefix) :] if s.startswith(prefix) else s
    return rest


### The system considers whether to terminate a line after the line is generated.
### So punctuation of a line could end up in the next line down.
def fix_final_lines_punctuation(lines):
    new_lines = []
    for i, line in enumerate(lines):
        if i > 0 and line[0] in PUNCTUATION:
            new_lines[-1] = new_lines[-1] + line[0]
            new_lines.append(line[1:].strip())
        else:
            new_lines.append(line.strip())
    return new_lines


def fix_final_lines_capitalization(lines):
    new_lines = []
    for line in lines:
        new_line = line[0].upper() + line[1:].lower()
        new_lines.append(new_line)
    return new_lines


### Figure out how to put two lines (strings) together.
### If force_break is True, then put a sentence break (period) between the two.
### Otherwise, try to figure out if there should be a period between.
def merge_lines(l1, l2, force_break=False):
    if len(l1) == 0:
        return l2
    elif len(l2) == 0 and force_break:
        return l1 + PERIOD
    elif len(l2) == 0:
        return l1
    elif not is_punctuation(l1[-1]) and force_break:
        return l1 + PERIOD + BLANK + l2
    else:
        return l1 + BLANK + l2


### Figure out which xlnet tokens are numbers
def xlnet_number_tokens():
    nums = []
    for i in range(len(xlnet_tokenizer.get_vocab())):
        s = xlnet_tokenizer.decode(i)
        if (
            len(s) > 0 and len(set(s).intersection(set(NUMBERS))) > 0
        ):  # s[0] in NUMBERS:
            nums.append((i, s))
    return nums


### Figure out which gpt tokens are numbers
def gpt_number_tokens():
    nums = []
    for i in range(len(gpt2_tokenizer.get_vocab())):
        s = gpt2_tokenizer.decode(i)
        if (
            len(s) > 0 and len(set(s).intersection(set(NUMBERS))) > 0
        ):  # s[0] in NUMBERS:
            nums.append((i, s))
    return nums


### Find all the numbers in xlnet and gpt vocabularies
XLNET_NUMBER_TOKENS = list(map(lambda x: x[0], xlnet_number_tokens()))
GPT_NUMBER_TOKENS = list(map(lambda x: x[0], gpt_number_tokens()))


### Print lyrics
def pretty_print(lines):
    print("---------")
    for line in lines:
        print(line)
    print("---------")


EXCLAIMS = ["oh!", "ah!", "yeah!"]


## If we don't have enough syllables, add some ohs ahs and yeahs
def add_filler(line, count, start):
    prefix = line[0:start]
    suffix = line[start:]
    filler = []
    for i in range(count):
        filler.append(random.choice(EXCLAIMS))
    return prefix + BLANK + BLANK.join(filler) + BLANK + suffix


## We get weird punctuation. Remove it
def remove_extra_punctuation(line):
    idx = len(line)
    for i in range(1, len(line)):
        pos = len(line) - i
        if line[pos] in PUNCTUATION:
            idx = pos
        else:
            break
    return line[0:idx]


### Merge two dictionaries.
### The dictionaries should contain lists as values.
def merge_dicts(dict1, dict2):
    if dict1 is None or dict2 is None:
        return {}
    new_dict = copy.deepcopy(dict1)
    for key in list(dict2.keys()):
        val_list = dict2[key]
        if key not in new_dict:
            new_dict[key] = []
        new_dict[key] += val_list
    return new_dict


### Count how many times each word occurs in a line.
def word_counts(line):
    counts = {}
    new_line = "".join(list(filter(lambda c: c not in PUNCTUATION, line)))
    for word in new_line.split():
        if word not in counts:
            counts[word] = 0
        counts[word] += 1
    return counts


### Get rid of any lines that uses a word more than once
def filter_lines(lines):
    return list(filter(lambda line: max(word_counts(line).values()) <= 1, lines))


### Make sure our lines don't get too long
def crop_line(line):
    enc = gpt2_tokenizer.encode(line)
    if len(enc) > MAX_CONTEXT_LENGTH:
        enc = enc[len(enc) - MAX_CONTEXT_LENGTH :]
    dec = gpt2_tokenizer.decode(enc)
    return dec

In [18]:
import re

def repeat_fn(line, all_lines, line_number, spec, args):
    target_syllables = args[0]
    ref_line = all_lines[line_number]
    ref_line = "".join(c for c in ref_line if not c in PUNCTUATION)
    if target_syllables == "all":
        return ref_line, all_lines, line_number, spec
    else:
        ref_line_words = ref_line.split()
        syllable_count = 0
        picked_words = []
        for word in reversed(ref_line_words):
            num_syllables = get_syllables_for_line(word)
            syllable_count = syllable_count + num_syllables
            picked_words.append(word)
            if syllable_count >= target_syllables:
                break
        return BLANK.join(reversed(picked_words)), all_lines, line_number, spec


def frepeat_fn(line, all_lines, line_number, spec, args):
    target_syllables = args[0]
    ref_line = all_lines[line_number]
    ref_line = "".join(c for c in ref_line if not c in PUNCTUATION)
    if target_syllables == "all":
        return ref_line, all_lines, line_number, spec
    else:
        ref_line_words = ref_line.split()
        syllable_count = 0
        picked_words = []
        for word in ref_line_words:
            num_syllables = get_syllables_for_line(word)
            syllable_count = syllable_count + num_syllables
            picked_words.append(word)
            if syllable_count >= target_syllables:
                break
        return BLANK.join(picked_words), all_lines, line_number, spec


def copy_fn(line, all_lines, line_number, spec, args):
    new_spec = {}
    line_to_copy = args[0]
    copied_line = all_lines[line_to_copy]
    new_all_lines = (
        all_lines[0 : line_number + 1] + [copied_line] + all_lines[line_number + 1 :]
    )
    for key in list(spec.keys()):
        val = spec[key]
        if key > line_number:
            new_spec[key + 1] = val
        else:
            new_spec[key] = val
    return "", new_all_lines, line_number, new_spec


PARSE_FUNCTIONS = {"repeat": repeat_fn, "copy": copy_fn, "frepeat": frepeat_fn}


def parse(line, all_lines, line_number, spec):
    done = False
    while not done:
        match = re.search(r"\{([\w]+)[ ]*([\w, ]*)\}", line)
        if match is None:
            done = True
        else:
            func = match.groups()[0]
            args = eval(match.groups()[1])
            if not isinstance(args, tuple):
                args = tuple([args])
            pre = line[0 : match.start()]
            mid, all_lines, line_number, spec = PARSE_FUNCTIONS[func](
                line, all_lines, line_number, spec, args
            )
            post = line[match.end() :]
            line = pre + mid + post
    return line, all_lines, line_number, spec


### Add post-processing information to each line
def post_process_lines(lines, spec):
    new_lines = []
    i = 0
    while i < len(lines):
        line = lines[i]
        if i in spec:
            pre, post = spec[i]
            if pre is not None:
                pre, lines, i, spec = (
                    parse(pre, lines, i, spec) if pre is not None else ""
                )
                line = lines[i]
            else:
                pre = ""
            if post is not None:
                post, lines, i, spec = (
                    parse(post, lines, i, spec) if post is not None else ""
                )
                line = lines[i]
            else:
                post = ""
            new_lines.append(pre + line + post)
        else:
            new_lines.append(line)
        i = i + 1
    return new_lines

In [19]:
import torch.nn
import torch.nn.functional as F


### Pick the word from the words list that is most similar to the context according to BERT
def pick_similar(context, words, history = []): 
    words = words[:]
    token_hash = {}
    for word in words:
        token_hash[tuple(gpt2_tokenizer.encode(word))] = None
    
    #prep_gpt()
    context_tokens = gpt2_tokenizer.encode(context) 
    prompt = torch.tensor([context_tokens]) # context put into the right shape
    prompt = prompt.to('cuda') if CUDA_AVAILABLE else prompt
    past = None 
    beams = []
    for i in range(10):
        current_beam = []
        for j in range(20):
            # Generate
            output, new_past = gpt2_model(prompt, past_key_values=past)
            # Top k filter: there are k real numbers and the rest are -inf
            logits = output[0][0][:]
            tokens = torch.multinomial(F.softmax(logits), 10)
            #token = tokens[0]
            for tok in tokens.tolist():
                word = gpt2_tokenizer.decode(tok)
                if not is_punctuation(word):
                    current_beam.append((tok, F.softmax(logits)))
                    prompt = torch.tensor([tok]).unsqueeze(0)
                    prompt = prompt.to('cuda')
                    past = new_past
                    break
            #if token == 13:
            #  break
            #else:
            #  current_beam.append((token, logits))
            #  prompt = torch.tensor([token]).unsqueeze(0)
            #  prompt = prompt.to('cuda')
            #  past = new_past
        beams.append(current_beam)
        # ASSERT: we have 10 beams of 20 tokens or less
        '''
        for b in beams:
        x = []
        for tok, ls in b:
            x.append(tok)
        print('---')
        print(GPT_TOKENIZER.decode(x))
        '''
    for key in list(token_hash.keys()):
        sum = 0
        for beam in beams:
            for i in range(len(beam)):
                for j, tok in enumerate(key):
                    if i+j < len(beam):
                        logits = beam[i+j][1]
                        sum = sum + logits[tok]
                if len(key) > 1:
                    sum = sum / len(key)
            #if len(beam) > 1:
            #  sum = sum / len(beam)
        if token_hash[key] is None:
            token_hash[key] = sum
        else:
            token_hash[key] = max(token_hash[key], sum)
    # ASSERT: Token_hash populated with good values
    vals = torch.stack(list(token_hash.values()))
    #soft = F.softmax(vals)
    #mask = soft > 0.0
    #num_nonzero = torch.sum(mask.int())
    vals = vals + -vals.min()
    #vals = F.softmax(vals)
    #vals = -vals
    #topk = torch.multinomial(soft, min(10, len(words), num_nonzero.item()), replacement=False).tolist()
    final_pick = None
    if vals.sum().item() > 0:
        topk = torch.multinomial(vals, vals.size()[0], replacement=False).tolist()
        print("Picking from:", list(map(lambda x: words[x], topk)), "given history", history)
        #pdb.set_trace()
        for idx in topk:
            if words[idx] not in history:
                final_pick = words[idx]
                break
    if final_pick is None and len(words) >= 1:
        final_pick = random.choice(words)
    if final_pick is None:
        # Uh oh we didn't find anything, probably because all the hits were in history
        sorted_words = sorted(words, key=lambda w: token_hash[tuple(gpt2_tokenizer.encode(w))], reverse=True)
        print("Second attempt", sorted_words, "history", history)
        final_pick = None
        for w in sorted_words:
            if w not in history:
                final_pick = w
                break
        if final_pick is None:
            weights = list(map(lambda w: -token_hash[tuple(gpt2_tokenizer.encode(w))], sorted_words))
            print("Third attempt", sorted_words, "history", history)
            final_pick = random.choices(sorted_words, weights)[0]
    print("Picking", final_pick)
    return final_pick

In [20]:
from wordfreq import word_frequency, top_n_list
import pronouncing

END_NO_WORDS = [
    "and",
    "to",
    "the",
    "a",
    "with",
    "an",
    "of",
    "as",
    "if",
    "is",
    "for",
    "or",
    "nor",
    "in",
    "we",
    "my",
    "from",
    "that",
    "your",
]

NO_RHYME_WORDS = [
    "francoise",
    "l'enfant",
    "un",
    "vanhove",
    "suhud",
    "re",
    "le",
    "tao",
    "mao",
    "lao",
    "petr",
    "chas",
    "rao",
    "raby",
    "rabey",
    "ia",
    "co",
    "uk",
    "a",
    "i",
    "the",
    "nba",
    "dna",
    "fda",
    "gta",
    "c'est",
    "les",
    "tor",
    "dvd",
    "labov",
    "stumm",
    "yu",
    "du",
    "mon",
    "aka",
    "de",
    "ne",
    "que",
    "ga",
    "si",
    "ap",
    "ta",
    "und",
    "des",
    "wa",
    "ka",
    "ra",
    "ba",
    "da",
    "aa",
    "ca",
    "va",
    "ja",
    "sa",
    "fa",
    "ou",
    "eu",
    "tu",
    "su",
    "fu",
    "wu",
    "dui",
    "der",
    "ib",
    "something",
    "jesus",
    "twentysomething",
    "schaab",
    "soo",
    "grier",
    "guangdong",
] + END_NO_WORDS

RHYME_VOCAB_TOP_N = 50000


### Remove some words from rhyme dictionary
def filter_rhyme(word):
    return (
        (PERIOD not in word)
        and ("-" not in word)
        and (len(word) > 1 or word == "a" or word == "I" or word == "i")
        and (word.lower() not in NO_RHYME_WORDS)
        and (word.lower() in top_n_list("en", RHYME_VOCAB_TOP_N))
    )


### Uniformly pick amongst top k most frequent perfect rhymes
def pick_perfect_rhyme(word, context=None, history=[]):
    # Remove unwanted letters
    word = "".join(list(filter(lambda c: c in STR_LETTERS, word)))
    # Get rhymes
    rhymes = pronouncing.rhymes(word)
    # Remove unwanted words
    rhymes = list(filter(lambda w: filter_rhyme(w), rhymes))
    # If there aren't any rhymes, return the current word
    if len(rhymes) == 0:
        return word
    else:
        # Make sure k is smaller than total number of rhymes available
        k = min(RHYME_K, len(rhymes))
        # Get word frequency of rhymes
        probs = list(map(lambda r: word_frequency(r, "en"), rhymes))
        probs_tensor = torch.tensor(probs)
        # Get top k
        vals, idxs = torch.topk(probs_tensor, k)
        if context is not None and len(context) > 0:
            candidates = [rhymes[x] for x in idxs.tolist()]
            pick = pick_similar(context, candidates, history)
            return pick
        else:
            # Pick uniformly
            r = random.randint(0, k - 1)
            return rhymes[idxs[r].item()]


### Pick best near rhyme from the near rhyme dictionary
def pick_near_rhyme(word, context=None, history=[]):
    global NEAR_RHYME_DICT
    if NEAR_RHYME_DICT is None:
        NEAR_RHYME_DICT = load_near_rhyme_dictionary(NEAR_RHYME_PATH)
    # Remove unwanted letters
    word = "".join(list(filter(lambda c: c in STR_LETTERS, word)))
    # Get rhymes
    near_rhymes = []
    # rhymes = []
    if word in NEAR_RHYME_DICT:
        near_rhymes = NEAR_RHYME_DICT[word]
    if len(near_rhymes) > 0:
        rhymes = near_rhymes
    # else:
    #  rhymes = pronouncing.rhymes(word)
    rhymes = list(set(near_rhymes + pronouncing.rhymes(word)))
    # Remove unwanted words
    rhymes = list(filter(lambda w: filter_rhyme(w), rhymes))
    # If there aren't any rhymes, return the current word
    if len(rhymes) == 0:
        return word
    else:
        # Make sure k is smaller than total number of rhymes available
        k = min(RHYME_K, len(rhymes))
        # Get word frequency of rhymes
        probs = list(map(lambda r: word_frequency(r, "en"), rhymes))
        probs_tensor = torch.tensor(probs)
        # Get top k
        vals, idxs = torch.topk(probs_tensor, k)
        if context is not None and len(context) > 0:
            candidates = [rhymes[x] for x in idxs.tolist()]
            pick = pick_similar(context, candidates, history)
            return pick
        else:
            # Pick uniformly
            r = random.randint(0, k - 1)
            return rhymes[idxs[r].item()]


### Pick rhyme, redirects to pick_perfect_rhyme or pick_near_rhyme
def pick_rhyme(word, perfect=True, context=None, history=[]):
    if perfect:
        return pick_perfect_rhyme(word, context=context, history=history)
    else:
        return pick_near_rhyme(word, context=context, history=history)

In [21]:
LOSS = torch.nn.CrossEntropyLoss(reduction="sum")


def interactive_pick_line(lines):
    result = None
    print("GENERATED CANDIDATES:")
    for i, line in enumerate(lines):
        print("[" + str(i) + "] " + line)
    inp = input("CHOOSE BY NUMBER (0-" + str(len(lines) - 1) + ") OR WRITE YOUR OWN: ")
    try:
        idx = int(inp)
        result = lines[idx]
    except ValueError:
        result = inp
    return result, INF


### Score a sentence, lower is better (cross entropy)

def score_sentence(sentence):
    # encode sentence
    prompt = gpt2_tokenizer.encode(sentence)
    # Set up x and y as shifted input
    x = torch.tensor([prompt[:-1]])
    y = torch.tensor(prompt[1:])
    if CUDA_AVAILABLE:
        x = x.to("cuda")
        y = y.to("cuda")
    # prep_gpt()
    # Measure the logits for loss
    output, new_past = gpt2_model(x, past_key_values=None)
    score = LOSS(output[0], y)
    return score


### Pick the best line. Run them all through the scoring function and pick the smallest
def pick_best_line(lines, context, history=[], interactive=False):
    if interactive:
        return interactive_pick_line(lines)
    else:
        # Remove duplicates
        modified_history = list(
            map(
                lambda s: remove_punctuation(s).encode("ascii", "ignore").lower(),
                history,
            )
        )
        filtered_lines = list(
            filter(
                lambda s: remove_punctuation(s).encode("ascii", "ignore").lower()
                not in modified_history,
                lines,
            )
        )
        lines = filtered_lines if len(filtered_lines) > 0 else lines
        # Score sentences
        scores = list(map(lambda l: score_sentence(merge_lines(context, l)), lines))
        scores_tensor = torch.tensor(scores, dtype=torch.float)
        # for i, line in enumerate(lines):
        #  print("SCORE", scores[i].item(), line)
        # Get smallest
        if len(lines) == 1:
            return lines[0], scores[0].item()
        elif GREEDY_PICK_LINE:
            vals, idxs = torch.topk(scores_tensor, 1, largest=False)
            idx = idxs[0].item()
            return lines[idx], vals[0].item()
        else:
            try:
                shifted = scores_tensor - min(scores_tensor)
                flipped = (max(shifted) - shifted).div(max(shifted))
                exped = flipped.div(PICK_LINE_TEMPERATURE).exp()
                idxs = torch.multinomial(exped, 1)
                idx = idxs[0].item()
            except:
                # Probably a div by zero
                # Usually caused when all the sentences are the same and thus have the same scores
                # Which means shifted is a tensor of 0s
                idx = 0
            print("PICK", lines[idx])
            return lines[idx], scores[idx].item()

In [22]:
MASK_IDX = 6  # MASK is token id 6
PERIOD_IDX = 9  # Period is token id 9
# Things I don't want generated
XLNET_NO_TOKENS = [
    6,
    7,
    8,
    0,
    1,
    2,
    3,
    4,
    5,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    2055,
    6490,
    26,
    97,
    167,
    225,
    4145,
    3158,
    17115,
    22891,
    17666,
    4538,
    1926,
    22788,
    2780,
    19348,
    16011,
] + XLNET_NUMBER_TOKENS
NINF = -float("Inf")  # Negative infinity
INF = float("Inf")  # Positive infinity


### Given a prompt, with one or more '<mask>' in it, fill the masks in with XLNET
### Can go forward or backward. Backward seems to work better?

def fill_line(prompt, backward=False):
    generated_tokens = []
    # Convert prompt into tokens
    input_ids = torch.tensor(
        xlnet_tokenizer.encode(prompt, add_special_tokens=False)
    ).unsqueeze(0)
    # mask out the places we want to predict
    perm_mask = torch.zeros(
        (1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float
    )
    masked = input_ids == MASK_IDX
    perm_mask = perm_mask + masked
    # The places we want to predict are...
    predicts = torch.nonzero(masked[0]).tolist()
    if backward:
        predicts = list(reversed(predicts))
    # Set up a diagonal where we want to predict, dim=0 is batch, dim=1 is each prediction
    target_mapping = torch.zeros(
        (1, len(predicts), input_ids.shape[1]), dtype=torch.float
    )
    for n, p in enumerate(predicts):
        target_mapping[0][n][p] = 1.0
    # prep_xlnet()
    if CUDA_AVAILABLE:
        input_ids = input_ids.to("cuda")
        perm_mask = perm_mask.to("cuda")
        target_mapping = target_mapping.to("cuda")

    # Fill one mask at a time until there are no places to fill (predicts is empty)
    while len(predicts) > 0:
        # Predict everything, but ignore all but the first prediction
        outputs = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)
        next_token_logits = outputs[
            0
        ]  # Output has shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]
        # Filter to top-k
        logits = top_k_top_p_filtering(next_token_logits[0][0], top_k=SAMPLE_K)
        # Sample from top-k
        samples = torch.multinomial(F.softmax(logits), SAMPLE_K)
        # Make sure we didn't predict a repetition and not in NO_TOKENS
        pos = torch.nonzero(target_mapping[0][0]).item()
        previous_token = input_ids[0][pos - 1].item()
        next_token = input_ids[0][pos + 1].item()
        token = None
        for i in range(SAMPLE_K):
            tok = samples[i].item()
            if (
                tok != previous_token
                and tok != next_token
                and tok not in XLNET_NO_TOKENS
            ):
                # Avoid ALL punctuation
                if not is_punctuation(xlnet_tokenizer.decode(tok)):
                    # avoid repeat tokens
                    word = xlnet_tokenizer.decode(tok)
                    if (
                        word.lower()
                        not in xlnet_tokenizer.decode(generated_tokens).lower()
                    ):
                        # avoid ending with weird words
                        # if word.lower() in END_NO_WORDS:
                        #  print('--', word.lower(), len(predicts))
                        if word.lower() not in END_NO_WORDS or len(predicts) > 1:
                            token = tok
                            generated_tokens.append(tok)
                            break
        if token is None:
            token = samples[0].item()
            generated_tokens.append(token)
        # insert the token into the input
        input_ids[0][pos] = token
        # print(XLNET_TOKENIZER.decode(input_ids[0]))
        # Mask out everything that needs to be masked
        perm_mask = torch.zeros(
            (1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float
        )
        perm_mask = perm_mask.to("cuda")
        masked = input_ids == 6
        perm_mask = perm_mask + masked
        # Update predicts, should be one less
        predicts = torch.nonzero(masked[0]).tolist()
        if backward:
            predicts = list(reversed(predicts))
        # Set up a diagonal where we want to predict, dim=0 is batch, dim=1 is each prediction
        target_mapping = torch.zeros(
            (1, len(predicts), input_ids.shape[1]), dtype=torch.float
        )
        for n, p in enumerate(predicts):
            target_mapping[0][n][p] = 1.0
        target_mapping = target_mapping.to("cuda")
    return xlnet_tokenizer.decode(input_ids[0])

In [23]:
def generate_rhyme_line(rhyme, context, target_syllables, terminate_line=True):
  good_tries = [] # runs with the exact number of syllables
  bad_tries = {}  # Runs with fewer syllables (dict is num_syllables: list of lines)
  # i is number of masks
  # j is number of tries
  for i in range(int(target_syllables*1.5)):
    for j in range(NUM_TRIES):
      ## Even tries have a period added at the end of the previous context
      #if j % 2 == 0 and len(context) > 0 and context[-1] not in PUNCTUATION and not no_terminate_previous:
      #  context_copy = context[:] + PERIOD
      #else:
      context_copy = context[:] + BLANK
      # This fixes things up
      context_copy = xlnet_tokenizer.decode(xlnet_tokenizer.encode(context_copy, add_special_tokens=False))
      # Make masks
      prompt = context_copy + BLANK + BLANK.join(['<mask>']*(i+1)) + BLANK + rhyme
      if terminate_line or j % 2 == 1:
        prompt = prompt + PERIOD
      # Fill masks
      filled_line = fill_line(prompt, backward=True)
      # Figure out how many syllables we added
      candidate = remove_prefix(filled_line, context_copy).strip() # the newly added line
      line_syllable_count = get_syllables_for_line(candidate) # Number of syllables in newly added line
      # Did we get a good run?
      if line_syllable_count == target_syllables:
        good_tries.append(candidate)
        print("CANDIDATE", candidate)
      elif line_syllable_count < target_syllables:
        # Bad tries are those that are too short. Store in dict by length
        if line_syllable_count not in bad_tries:
          bad_tries[line_syllable_count] = []
        bad_tries[line_syllable_count].append(candidate)
  return good_tries, bad_tries

In [24]:
GPT_NO_TOKENS = [
    59,
    50256,
    1,
    6,
    7,
    8,
    12,
    14,
    26,
    58,
    59,
    60,
    62,
    90,
    92,
    2503,
    3,
    1906,
    14988,
    26391,
    4023,
    338,
    40578,
    14231,
    14036,
    15466,
    525,
] + GPT_NUMBER_TOKENS



def generate_non_rhyme_line(context, target_syllables):
    generated_tokens = []  # Tokens generated along the way
    good_tries = []  # results with the correct number of syllables
    past_syllables = get_syllables_for_line(
        context
    )  # How many syllables in the context
    # prep_gpt()
    # j is number of tries
    for j in range(NUM_TRIES):
        new_syllables = 0  # How many new syllables were produced
        # Odd tries add a period to the context
        # if j % 2 == 0 and len(context) and context[-1] not in PUNCTUATION and not no_terminate_previous:
        #  context_copy = context[:] + PERIOD
        # else:
        context_copy = context[:] + BLANK
        # Encode the context
        generated = gpt2_tokenizer.encode(context_copy)  # Used to collect up tokens
        prompt = torch.tensor([generated])  # context put into the right shape
        prompt = prompt.to("cuda") if CUDA_AVAILABLE else prompt
        past = None  # Initially we don't have any history
        # Generate until we get enough syllables
        previous_token = generated[-1] if len(generated) > 0 else None
        count = 0
        while (
            new_syllables < target_syllables and count < 1000
        ):  # break loop if too many iterations
            # Generate
            output, new_past = gpt2_model(prompt, past_key_values=past)
            # Top k filter: there are k real numbers and the rest are -inf
            logits = top_k_top_p_filtering(output[0][0], top_k=SAMPLE_K)
            tokens = torch.multinomial(F.softmax(logits), SAMPLE_K)
            # Pick the first one from the top k that doesn't produce too many syllables
            for tok in tokens.tolist():
                # How many syllables do we have total?
                line_syllables = get_syllables_for_line(
                    gpt2_tokenizer.decode(generated + [tok])
                )
                # Have we gone over? Or generated a NO_TOKEN?
                if (
                    line_syllables <= target_syllables + past_syllables
                    and tok not in GPT_NO_TOKENS
                ):
                    # We are good
                    word = gpt2_tokenizer.decode(tok)  # The new word
                    # Don't allow ANY punctuation
                    if not is_punctuation(word):
                        if (
                            word.lower()
                            not in gpt2_tokenizer.decode(generated_tokens).lower()
                        ):
                            generated_tokens.append(tok)
                            # Add new word to generated
                            generated.append(tok)
                            # Prep for the next run
                            prompt = torch.tensor([tok]).unsqueeze(0)
                            prompt = prompt.to("cuda")
                            past = new_past
                            new_syllables = line_syllables - past_syllables
                            break
            count = count + 1
        candidate = remove_prefix(gpt2_tokenizer.decode(generated), context).strip()
        print("CANDIDATE", candidate)
        good_tries.append(candidate)
    return good_tries, None

In [25]:
def generate_terminal_non_rhyme_line(context, target_syllables):
    good_tries = []  # runs with the exact number of syllables
    bad_tries = {}  # Runs with fewer syllables (dict is num_syllables: list of lines)
    # i is number of masks
    # j is number of tries
    for i in range(int(target_syllables * 1.5)):
        for j in range(NUM_TRIES):
            # Even tries have a period added at the end of the previous context
            # if j % 2 == 0 and len(context) > 0 and context[-1] not in PUNCTUATION and not no_terminate_previous:
            #  context_copy = context[:] + PERIOD
            # else:
            context_copy = context[:] + BLANK
            # This fixes things up
            context_copy = xlnet_tokenizer.decode(
                xlnet_tokenizer.encode(context_copy, add_special_tokens=False)
            )
            # Make masks
            prompt = (
                context_copy + BLANK + BLANK.join(["<mask>"] * (i + 1)) + PERIOD
            )  # Different from generate_rhyme_line
            # Fill masks
            filled_line = fill_line(
                prompt, backward=False
            )  # Forward instead of backward
            # Figure out how many syllables we added
            candidate = remove_prefix(
                filled_line, context_copy
            ).strip()  # the newly added line
            line_syllable_count = get_syllables_for_line(
                candidate
            )  # Number of syllables in newly added line
            # Did we get a good run?
            if line_syllable_count == target_syllables:
                good_tries.append(candidate)
                print("CANDIDATE", candidate)
            elif line_syllable_count < target_syllables:
                # Bad tries are those that are too short. Store in dict by length
                if line_syllable_count not in bad_tries:
                    bad_tries[line_syllable_count] = []
                bad_tries[line_syllable_count].append(candidate)
    return good_tries, bad_tries

In [26]:
import pdb

def run(context, scheme, rhyme_dict, use_near_rhymes = False, post = None, 
        recontextualize = False, interactive = False):
  ### SETUP #####################################
  lines = []    # Store the lyrics lines
  segments = [] # Store each segment separately
  # Set up the context
  context = context[:].strip()
  original_context = context[:]
  # Set up rhyme history
  rhyme_history = [] # The history of words used for rhymes
  # original context shouldn't have punctuation at the end, but the first prompt to the neural net should
  if len(context) >0 and context[-1] not in PUNCTUATION:
    context = context + PERIOD
  if len(original_context) > 0 and original_context[-1] in PUNCTUATION:
    original_context = original_context[:-1]
  # copy the rhyme_dict because we will be adding to it
  rhyme_dict = copy.deepcopy(rhyme_dict)
  ### ITERATE THROUGH SCHEME #####################
  # An entry can be tuples or lists of tuples. 
  # Easiest thing is to nest tuples into lists and treat everything the same
  for n, entry in enumerate(scheme):
    ### RECONTEXTUALIZE ##########################
    # Insert the original context into the history/context at sentence breaks
    if n > 0 and recontextualize:
      last_period = context.rfind(PERIOD)
      last_contextualization = context.rfind(original_context)
      if last_period >= 0 and last_contextualization >= 0 and last_contextualization + len(original_context) - 1 != last_period - 1:
        context = merge_lines(merge_lines(context[:last_period+1], original_context, force_break=True), context[last_period+1:], force_break=True)
    ### ITERATE THROUGH LINE SPEC #################
    current_line = []   # The line currently being worked on. May be made of several segments
    # if line is a single segment, nest it
    if isinstance(entry, tuple):
      entry = [entry]
    # A line has 1 or more tuples of the form (target_syllables, rhyme_index)
    for segment_num, segment in enumerate(entry):
      is_end_segment = (segment_num == len(entry) - 1)
      target_syllables = segment[0]
      rhyme_idx = segment[1]
      cmd = None
      goods = []    # lines with the right number of syllables
      shorts = {}   # lines with too few syllables
      new_context = None # The complete lyrics after a new line is added
      new_syllables = 0 # The number of new syllables added to lyrics
      terminate_segment = False
      ### PARSE SPECIAL COMMANDS ##################
      # We have commands to parse
      if len(segment) > 2:
        cmd = segment[2]
        terminate_segment = (cmd == ':end')
      print("LINE", n, "SEGMENT", segment_num, '(end)' if is_end_segment else '', "TARGET SYLLABLES", target_syllables, "RHYME INDEX", rhyme_idx, "COMMAND", cmd)
      # Check if we are filling in a partial line (when there is a rhyme or when there is a given verbatim string)
      if isinstance(rhyme_idx, str) or rhyme_idx in rhyme_dict:
        ### USE RHYME INDEX ########################
        # We are filling a line
        end_targets = [] # The line should end in this word (or words)
        ### RHYME INDEX IS A STRING ####
        if isinstance(rhyme_idx, str):
          # Use a verbatim string
          end_targets = [rhyme_idx] # rhyme_idx is actually a string
          # If target_syllables < 0 then set it to the exact number of syllables in the rhyme_idx string
          if target_syllables < 0:
            target_syllables = get_syllables_for_line(rhyme_idx)
        ### PICK A RHYME WORD ####
        else:
          # Get some rhyming words
          # but first fix the dictionary if there is a single word instead of a list
          if not isinstance(rhyme_dict[rhyme_idx], list) and not isinstance(rhyme_dict[rhyme_idx], tuple):
            rhyme_dict[rhyme_idx] = [rhyme_dict[rhyme_idx]]
          # Now get some words
          if rhyme_idx not in rhyme_dict:
            pdb.set_trace()
          end_targets = [pick_rhyme(w, perfect=not use_near_rhymes, context=context, history=rhyme_history) for w in rhyme_dict[rhyme_idx]]
        # There could be more than one possible rhyme target
        ### GENERATE #####
        for end_target in end_targets:
          print("RHYME TARGET", end_target)
          # Check to see if the end_target already fills up all of our syllable length
          end_target_syllables = get_syllables_for_line(end_target)
          if end_target_syllables < target_syllables:
            # Generate new lines
            meet_target_syllables, too_short = generate_rhyme_line(end_target, context, target_syllables, terminate_line=terminate_segment)
            goods = goods + meet_target_syllables
            shorts = merge_dicts(shorts, too_short)
          else:
            # Just copy the end target because that is all the syllables we need
            goods.append(end_target)
      else:
        ### NON-RHYME SEGMENT ###############################
        # We are generating a new line unconstrained
        # Generate new line
        if terminate_segment:
          # We've determined this segment must be terminal.
          match_target_syllables1, shorts = generate_terminal_non_rhyme_line(context, target_syllables)
          goods = match_target_syllables1
        else:
          # Segment does not need to be terminal
          match_target_syllables1, _ = generate_non_rhyme_line(context, target_syllables)
          match_target_syllables2 =  []
          if is_end_segment:
            match_target_syllables2, shorts = generate_terminal_non_rhyme_line(context, target_syllables)
          goods = match_target_syllables1 + match_target_syllables2
      ### PICK BEST CANDIDATE ############################################
      results = []
      if len(goods) > 0:
        results = goods
      else:
        longest_key = max(list(shorts.keys()))
        results = shorts[longest_key]
      best_continuation, score = pick_best_line(results, context, history=segments, interactive=interactive)
      new_syllables = get_syllables_for_line(best_continuation)
      ### UPDATE RHYME DICTIONARY AND RHYME HISTORY ####################################
      # If we are using near rhymes, store the near rhyme as possible target for future lines
      if not isinstance(rhyme_idx, str) and rhyme_idx >=0:
        # get end word
        continuation = ''.join(list(filter(lambda c: c in STR_LETTERS or c == BLANK, best_continuation)))
        split_continuation = continuation.split(BLANK)
        end_word = split_continuation[-1]
        if end_word[-1] in PUNCTUATION:
          end_word = end_word[:-1]
        # Make a new entry in rhyme_dict
        if (rhyme_idx not in rhyme_dict) or (rhyme_idx in rhyme_dict and use_near_rhymes):
          if rhyme_idx not in rhyme_dict:
            rhyme_dict[rhyme_idx] = []
          rhyme_dict[rhyme_idx].append(end_word)
        # Add to rhyme history
        rhyme_history.append(end_word)
      ### DO SOME FIXING #################################################
      # Fill in if we don't have enough syllables
      if new_syllables < target_syllables:
        rest = add_filler(best_continuation, target_syllables - new_syllables, 0)
      ### SET UP CONTEXT FOR NEXT ITERATION ##########################
      context = merge_lines(context, best_continuation)
      print('CONTEXT', context)
      current_line.append(best_continuation)
      segments.append(best_continuation)
      # get ready for the next iteration
      context = crop_line(context)
      previous_is_end_segment = is_end_segment
    ### DONE WITH LINE #################################################
    # put the current line together
    lines.append(BLANK.join(fix_final_lines_punctuation(current_line)))
  ### DONE WITH SPEC ###################################################
  #lines = fix_final_lines_punctuation(lines)
  lines = post_process_lines(lines, post)
  lines = fix_final_lines_capitalization(lines)
  return lines

In [27]:
def extract_scheme(lines, use_near_rhymes = True):
  syllables = []
  ends = []
  rhyme_lines = {}
  # Compute syllables per line
  for line in lines:
    line = line.strip()
    num_syl = get_syllables_for_line(line)
    syllables.append(num_syl)
    ends.append(line[-1] == '.')
  # Figure out which lines rhyme, piecewise
  for i, line1 in enumerate(lines):
    line1 = line1.strip()
    words1 = line1.split()
    last1 = words1[-1]
    for j, line2 in enumerate(lines):
      if i != j:
        line2 = line2.strip()
        words2 = line2.split()
        last2 = words2[-1]
        if perfect_rhyme(last1, last2) or (use_near_rhymes and near_rhyme(last1, last2)):
          if i not in rhyme_lines:
            rhyme_lines[i] = []
          if j not in rhyme_lines:
            rhyme_lines[j] = []
          if j not in rhyme_lines[i]:
            rhyme_lines[i].append(j)
          if i not in rhyme_lines[j]:
            rhyme_lines[j].append(i)
  # Gather up all rhyming lines
  for l1 in list(rhyme_lines.keys()):
    for l2 in rhyme_lines[l1]:
      for l3 in list(rhyme_lines[l2]):
        if l3 not in rhyme_lines[l1]:
          rhyme_lines[l1].append(l3)
  # reduce to unique sets
  rhyme_sets = []
  for l in list(rhyme_lines.values()):
    rhyme_set = set(l)
    if rhyme_set not in rhyme_sets:
      rhyme_sets.append(rhyme_set) 
  # Assign rhyme indexes to lines
  rhyme_idxs = {}
  for n, s in enumerate(rhyme_sets):
    for l in s:
      rhyme_idxs[l] = n
  # Build schema
  schema = []
  for n in range(len(lines)):
    segment = None
    syl_num = syllables[n]
    rhyme_idx = rhyme_idxs[n] if n in rhyme_idxs else -1
    if ends[n]:
      segment = (syl_num, rhyme_idx, ':end')
    else:
      segment = (syl_num, rhyme_idx)
    schema.append(segment)
  return schema

In [28]:
rhyme_dict = {}   # Rhyming lines are indicated by number. What word should certain lines rhyme with?
post = {}         # Post-processing scheme

# BEAT IT, MICHAEL JACKSON
scheme = [
#They told him don't you ever come around here
          (11, 1), #0
#Don't want to see your face, you better disappear
          [(6, -1), (6, 1)], #1
#The fire's in their eyes and their words are really clear
          [(5, -1), (7, 1)], #2
#So beat it, just beat it
          (3, 2, ':end'), #3
#You better run, you better do what you can
          (11, 3), #4
#Don't want to see no blood, don't be a macho man
          [(6, -1), (6, 3)], #5
#You want to be tough, better do what you can
          [(5, -1), (6, 3)], #6
#So beat it, but you want to be bad
          [(3, 2), (6, -1)], #7
#Just beat it, beat it, beat it, beat it
          (3, 2, ':end'), #8
#No one wants to be defeated
          (8, 2), #9
#Showin' how funky and strong is your fight
          (10, 4), #10
#It doesn't matter who's wrong or right
          (9, 4), #11
#Just beat it, beat it
          (3, 2), #12
#Just beat it, beat it
          (3, 2), #13
#Just beat it, beat it
          (3, 2), #14
#Just beat it, beat it
          (3, 2, ':end') #15
]
#rhyme_dict[1] = ['here']
post[3] = (None, ' just {repeat 2}.')
post[8] = (None, ' ({repeat 2}), {repeat 2}, ({repeat 2})')
post[12] = (None, ' ({repeat 2})')
post[13] = (None, ' ({repeat 2})')
post[14] = (None, ' ({repeat 2})')
post[15] = (None, ' ({repeat 2}), oooooh')


In [29]:
### Top-k most frequent rhymes
RHYME_K = 50                          
# for top-k sampling of tokens from language models
SAMPLE_K = 40                         
# How many times to try a line
NUM_TRIES = 6                         
### Always pick the best scoring line?
GREEDY_PICK_LINE = False
### temperature for picking different possible options for lines. Set to 1 to be most permissive. 
### Set closer to zero to be more conservative and prefer the best scoring line more.
PICK_LINE_TEMPERATURE = 0.1
### Setting this too high can cause memory problems
MAX_CONTEXT_LENGTH = 125

In [30]:
context = "explain calculus differentiation rules"

In [31]:
RUN_ADVANCED_MODE = True #@param {type:"boolean"}
use_near_rhymes = True #@param {type:"boolean"}
recontextualize = False #@param {type:"boolean"}
interactive = False #@param {type:"boolean"}

# Run the generator
if RUN_ADVANCED_MODE:
  lines = run(context, scheme, rhyme_dict, 
              use_near_rhymes=use_near_rhymes, 
              post=post, 
              recontextualize=recontextualize, 
              interactive=interactive)
  # Print the outcome
  pretty_print(lines)


LINE 0 SEGMENT 0 (end) TARGET SYLLABLES 11 RHYME INDEX 1 COMMAND None


AttributeError: 'str' object has no attribute 'dim'

In [1]:
def syllable_count(word):
    count = 0
    vowels = "aeiouyAEIOUY"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count



In [8]:
syllable_count(input("Put in a word"))

1